In [28]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma



In [29]:
from dotenv import load_dotenv
#Ce module sert à lire les fichiers .env
load_dotenv()

False

In [30]:
import pandas as pd
books=pd.read_csv("cleaned_books.csv")
books

isbn13      isbn10  \
0     9780002005883  0002005883   
1     9780002261982  0002261987   
2     9780006178736  0006178731   
3     9780006280897  0006280897   
4     9780006280934  0006280935   
...             ...         ...   
5192  9788172235222  8172235224   
5193  9788173031014  8173031010   
5194  9788179921623  817992162X   
5195  9788185300535  8185300534   
5196  9789027712059  9027712050   

                                                  title  \
0                                                Gilead   
1                                          Spider's Web   
2                                        Rage of angels   
3                                        The Four Loves   
4                                   The Problem of Pain   
...                                                 ...   
5192                                  Mistaken Identity   
5193                                Journey to the East   
5194  The Monk Who Sold His Ferrari: A Fable About F...   
5195                                          I Am that   
5196                           The Berlin Phenomenology   

                                           authors  \
0                               Marilynne Robinson   
1                  Charles Osborne;Agatha Christie   
2                                   Sidney Sheldon   
3                              Clive Staples Lewis   
4                              Clive Staples Lewis   
...                                            ...   
5192                              Nayantara Sahgal   
5193                                 Hermann Hesse   
5194                                  Robin Sharma   
5195  Sri Nisargadatta Maharaj;Sudhakar S. Dikshit   
5196                 Georg Wilhelm Friedrich Hegel   

                         categories  \
0                           Fiction   
1     Detective and mystery stories   
2                           Fiction   
3                    Christian life   
4                    Christian life   
...                             ...   
5192        Indic fiction (English)   
5193              Adventure stories   
5194               Health & Fitness   
5195                     Philosophy   
5196                        History   

                                              thumbnail  \
0     http://books.google.com/books/content?id=KQZCP...   
1     http://books.google.com/books/content?id=gA5GP...   
2     http://books.google.com/books/content?id=FKo2T...   
3     http://books.google.com/books/content?id=XhQ5X...   
4     http://books.google.com/books/content?id=Kk-uV...   
...                                                 ...   
5192  http://books.google.com/books/content?id=q-tKP...   
5193  http://books.google.com/books/content?id=rq6JP...   
5194  http://books.google.com/books/content?id=c_7mf...   
5195  http://books.google.com/books/content?id=Fv_JP...   
5196  http://books.google.com/books/content?id=Vy7Sk...   

                                            description  published_year  \
0     A NOVEL THAT READERS and critics have been eag...          2004.0   
1     A new 'Christie for Christmas' -- a full-lengt...          2000.0   
2     A memorable, mesmerizing heroine Jennifer -- b...          1993.0   
3     Lewis' work on the nature of love divides love...          2002.0   
4     "In The Problem of Pain, C.S. Lewis, one of th...          2002.0   
...                                                 ...             ...   
5192  On A Train Journey Home To North India After L...          2003.0   
5193  This book tells the tale of a man who goes on ...          2002.0   
5194  Wisdom to Create a Life of Passion, Purpose, a...          2003.0   
5195  This collection of the timeless teachings of o...          1999.0   
5196  Since the three volume edition ofHegel's Philo...          1981.0   

      average_rating  num_pages  ratings_count  description_length  \
0               3.85      247.0          361.0                 NaN   
1              

In [141]:
books["tagged_description"].to_csv("tagged_description.csv", sep=",", index=False, header=False)

In [144]:
# Charger le fichier texte
raw_document =TextLoader("tagged_description.csv", encoding="utf-8").load()
# Créer un splitter qui découpe chaque document en morceaux de 50 caractères, sans chevauchement
text_splitter = CharacterTextSplitter(chunk_size=1, chunk_overlap=0, separator="\n")
# Découper les documents
documents=text_splitter.split_documents(raw_document)
#documents est une liste d’objets de type Document (de LangChain).
#Chaque objet Document contient :le texte découpé (page_content)
documents[1]

Created a chunk of size 1170, which is longer than the specified 1
Created a chunk of size 1216, which is longer than the specified 1
Created a chunk of size 375, which is longer than the specified 1
Created a chunk of size 311, which is longer than the specified 1
Created a chunk of size 483, which is longer than the specified 1
Created a chunk of size 484, which is longer than the specified 1
Created a chunk of size 962, which is longer than the specified 1
Created a chunk of size 188, which is longer than the specified 1
Created a chunk of size 845, which is longer than the specified 1
Created a chunk of size 296, which is longer than the specified 1
Created a chunk of size 197, which is longer than the specified 1
Created a chunk of size 881, which is longer than the specified 1
Created a chunk of size 1090, which is longer than the specified 1
Created a chunk of size 1191, which is longer than the specified 1
Created a chunk of size 306, which is longer than the specified 1
Create

Document(metadata={'source': 'tagged_description.csv'}, page_content='"9780002261982:A new \'Christie for Christmas\' -- a full-length novel adapted from her acclaimed play by Charles Osborne Following BLACK COFFEE and THE UNEXPECTED GUEST comes the final Agatha Christie play novelisation, bringing her superb storytelling to a new legion of fans. Clarissa, the wife of a Foreign Office diplomat, is given to daydreaming. \'Supposing I were to come down one morning and find a dead body in the library, what should I do?\' she muses. Clarissa has her chance to find out when she discovers a body in the drawing-room of her house in Kent. Desperate to dispose of the body before her husband comes home with an important foreign politician, Clarissa persuades her three house guests to become accessories and accomplices. It seems that the murdered man was not unknown to certain members of the house party (but which ones?), and the search begins for the murderer and the motive, while at the same ti

In [143]:
from langchain_community.embeddings import HuggingFaceEmbeddings
#L’embedding = le modèle qui fabrique les vecteurs.
db_books = Chroma.from_documents(documents, embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"))

In [145]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k = 10)
docs

[Document(id='8c47198d-d352-44cc-92cb-4ca6616ca3c1', metadata={'source': 'tagged_description.csv'}, page_content='"9780786808069:Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience."'),
 Document(id='1643b3d2-19a1-4e67-9f8e-f2eee05d739d', metadata={'source': 'tagged_description.csv'}, page_content='"9780786808069 :Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience."'),
 Document(id='f85c0eea-32aa-4928-88d4-65dbe18f3658', metadata={'source': 'taggedd_description.csv'}, page_content='"9780786808069 :Children will discover the exciting world of their own backyard in this in

In [148]:
isbn_str = docs[1].page_content.split(":", 1)[0].strip().replace('"', '')
books[books["isbn13"] == int(isbn_str)]

isbn13      isbn10                                title  \
3747  9780786808069  0786808063  Baby Einstein: Neighborhood Animals   

                                authors        categories  \
3747  Marilyn Singer;Julie Aigner-Clark  Juvenile Fiction   

                                              thumbnail  \
3747  http://books.google.com/books/content?id=X9a4P...   

                                            description  published_year  \
3747  Children will discover the exciting world of t...          2001.0   

      average_rating  num_pages  ratings_count  description_length  \
3747            3.89       16.0          180.0                 NaN   

                       title_and_subtitle  \
3747  Baby Einstein: Neighborhood Animals   

                                     tagged_description  
3747  9780786808069:Children will discover the excit...

In [ ]:
def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 10,
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k = 50)

    books_list = []

    for i in range(0, len(recs)):
        isbn = recs[i].page_content.split(":", 1)[0].strip().replace('"', '')
        books_list.append(int(isbn))

    return books[books["isbn13"].isin(books_list)]
# isin est une méthode qui vérifie pour chaque élément de la colonne "isbn13" si cet élément est dans la liste books_list.

In [157]:
retrieve_semantic_recommendations("A book to teach children about beach", top_k=5)

c:\Users\Pc\Desktop\airecoommandation\venv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


isbn13      isbn10                                   title  \
30    9780006646006  000664600X                      Ocean Star Express   
407   9780064404419  0064404412                      The Rainbow People   
938   9780156949606  0156949601                               The Waves   
3215  9780689862465  0689862466                             The Octopus   
3442  9780744578263  0744578264                     The Sea-thing Child   
3747  9780786808069  0786808063     Baby Einstein: Neighborhood Animals   
3749  9780786808380  0786808381                   Baby Einstein: Babies   
3765  9780786819119  0786819111  Baby Einstein: Water, Water Everywhere   
4291  9780972851725  0972851720                 H.G. Wells' Floor Games   
5157  9781932416374  1932416374     The Very Persistent Gappers of Frip   

                                authors            categories  \
30             Mark Haddon;Peter Sutton      Juvenile Fiction   
407                        Laurence Yep      Juvenile Fiction   
938                      Virginia Woolf  Experimental fiction   
3215   Cynthia Rylant;Preston McDaniels      Juvenile Fiction   
3442                      Russell Hoban                 Birds   
3747  Marilyn Singer;Julie Aigner-Clark      Juvenile Fiction   
3749                 Julie Aigner-Clark      Juvenile Fiction   
3765                 Disney Book Group,      Juvenile Fiction   
4291      H. G. Wells;Barbara A. Turner    Games & Activities   
5157         George Saunders;Lane Smith      Juvenile Fiction   

                                              thumbnail  \
30    http://books.google.com/books/content?id=I2QZA...   
407   http://books.google.com/books/content?id=5AHwq...   
938   http://books.google.com/books/content?id=7HEM1...   
3215  http://books.google.com/books/content?id=8B8-C...   
3442  http://books.google.com/books/content?id=CEt9O...   
3747  http://books.google.com/books/content?id=X9a4P...   
3749  http://books.google.com/books/content?id=jv4NA...   
3765  http://books.google.com/books/content?id=tuAdA...   
4291  http://books.google.com/books/content?id=4Pglf...   
5157  http://books.google.com/books/content?id=xYJOP...   

                                            description  published_year  \
30    Joe and his parents are enjoying a summer holi...          2002.0   
407   "Culled from 69 stories collected in a [1930s]...          1992.0   
938   Against the background of the sea, the author ...          1959.0   
3215  Whistler and Lila, two of the mice children wh...          2005.0   
3442  Washed up on the beach during a storm, the sea...          2000.0   
3747  Children will discover the exciting world of t...          2001.0   
3749  Introduce your babies to birds, cats, dogs, an...          2002.0   
3765  Charming illustrations and playful rhythmic ve...          2003.0   
4291  Suggests some of the games that can be created...          2004.0   
5157  A novel set in the seaside village of Frip fol...          2006.0   

      average_rating  num_pages  ratings_count  description_length  \
30              3.50       32.0            1.0                 NaN   
407             3.75      208.0          202.0                 NaN   
938             4.14      297.0        19210.0                 NaN   
3215            4.16       64.0          117.0                 NaN   
3442            3.79       48.0          100.0                 NaN   
3747            3.89       16.0          180.0                 NaN   
3749            4.03       20.0           29.0                 NaN   
3765            3.70       10.0           77.0                 NaN   
4291            3.41      110.0           47.0                 NaN   
5157            4.11       84.0         2317.0                 NaN   

                                     title_and_subtitle  \
30                                   Ocean Star Express   
407                                  The Rainbow People   
938                                           T